In [1]:
# initial settings
#Note the installation of the forecast packages takes a while
library_path <- paste(getwd(), "packages",sep="/")
dir.create(library_path,showWarnings = FALSE)
.libPaths(library_path)

if(!require(tidyverse)){
    install.packages("tidyverse")
    library(tidyverse)
}
if(!require(repr)){
    install.packages("repr")
    library(repr)
}
if(!require(rpart)){
    install.packages("rpart")
    library(rpart)
}
if(!require(rpart.plot)){
    install.packages("rpart.plot")
    library(rpart.plot)
}
if(!require(caret)){
    install.packages("caret")
    library(caret)
}
if(!require(precrec)){
    install.packages("precrec")
    library(precrec)
}
if(!require(e1071)){
    install.packages("e1071")
    library(e1071)
}
if(!require(MASS)){
    install.packages("e1071")
    library(e1071)
}
if(!require(glmulti)){
    install.packages("e1071")
    library(e1071)
}

#library(tidyverse)
#library(tree)
#library(rpart)
#library(rpart.plot)
#library(caret)
#library(MASS)
#library(glmulti)

#install.packages('precrec',lib='.', verbose=TRUE)
#library(precrec,lib.loc='.')


# Plot size depending on your screen resolution to 6 x 9
options(repr.plot.width=9, repr.plot.height=6)


Loading required package: tidyverse

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.3.5     ✔ purrr   0.3.4
✔ tibble  3.1.6     ✔ dplyr   1.0.8
✔ tidyr   1.2.0     ✔ stringr 1.4.0
✔ readr   2.1.2     ✔ forcats 0.5.1

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

Loading required package: repr

Loading required package: rpart

Loading required package: rpart.plot

Loading required package: caret

Loading required package: lattice


Attaching package: ‘caret’


The following object is masked from ‘package:purrr’:

    lift


Loading required package: precrec

Loading required package: e1071

Loading required package: MASS


Attaching package: ‘MASS’


The following object is masked from ‘package:dplyr’:

    select


Loading required package: glmulti

Warning message in library(package, lib.loc = lib.loc, character.only = TRUE, lo


The downloaded binary packages are in
	/var/folders/hj/vv2hsj8534d1nypkg8lzwy_w0000gn/T//Rtmpnt6AYa/downloaded_packages


# Welcome to Day 2 ML Workshop


### Exercise 1: Build a Logistic Regression for Classification


Read in the `eBayAuctions.csv` file. This dataset contains information on 1972 auctions transacted on eBay.com during May-June 2004. The goal is to use these data to build a model that will distinguish competitive auctions from non-competitive ones. A competitive auction is defined as an auction with at least two bids placed on the item being
auctioned. 

The data include variables that describe the item (auction category), the seller (his or her eBay rating), and the auction terms that the seller selected (auction duration, opening price, currency, day of week of auction close). In addition, we have the price at which the auction closed. The goal is to predict whether or not an auction of interest
will be competitive.

##### Data preprocessing.  
Create dummy variables for the categorical predictors.
These include Category (18 categories), Currency (USD, GBP, Euro), EndDay (Monday-Sunday), and Duration (1, 3, 5, 7, or 10 days).

In [2]:
#load the data
ebay.df <- read.csv("ebayAuctions.csv", stringsAsFactors = FALSE)
head(ebay.df)

,Category,currency,sellerRating,Duration,endDay,ClosePrice,OpenPrice,Competitive.
,<chr>,<chr>,<int>,<int>,<chr>,<dbl>,<dbl>,<int>
1,Music/Movie/Game,US,3249,5,Mon,0.01,0.01,0
2,Music/Movie/Game,US,3249,5,Mon,0.01,0.01,0
3,Music/Movie/Game,US,3249,5,Mon,0.01,0.01,0
4,Music/Movie/Game,US,3249,5,Mon,0.01,0.01,0
5,Music/Movie/Game,US,3249,5,Mon,0.01,0.01,0
6,Music/Movie/Game,US,3249,5,Mon,0.01,0.01,0


Data prepocessing for categorical variable:
- Create pivot tables for the mean of the binary outcome (Competitive?) as a function of the various categorical variables (use the original variables, not the dummies). Use the information in the tables to reduce the number of dummies that will be used in the model. For example, categories that appear most similar with respect to the distribution of competitive auctions could be combined.

In [3]:
data.frame(tapply(ebay.df$Competitive., ebay.df$Category, mean))
data.frame(tapply(ebay.df$Competitive., ebay.df$currency, mean))
data.frame(tapply(ebay.df$Competitive., ebay.df$endDay, mean))
data.frame(tapply(ebay.df$Competitive., ebay.df$Duration, mean))

,tapply.ebay.df.Competitive...ebay.df.Category..mean.
,<dbl>
Antique/Art/Craft,0.5649718
Automotive,0.3539326
Books,0.5000000
Business/Industrial,0.6666667
Clothing/Accessories,0.5042017
Coins/Stamps,0.2972973
Collectibles,0.5774059
Computer,0.6666667
Electronics,0.8000000


,tapply.ebay.df.Competitive...ebay.df.currency..mean.
,<dbl>
EUR,0.5515947
GBP,0.6870748
US,0.5193498


,tapply.ebay.df.Competitive...ebay.df.endDay..mean.
,<dbl>
Fri,0.4668990
Mon,0.6733577
Sat,0.4273504
Sun,0.4852071
Thu,0.6039604
Tue,0.5321637
Wed,0.4800000


,tapply.ebay.df.Competitive...ebay.df.Duration..mean.
,<dbl>
1,0.5217391
3,0.4507042
5,0.6866953
7,0.4891417
10,0.5445545


The categories "Business/Industrials" and "Computers" have same average of the binary variable "Competitive." This means that the percentage of competitive auctions in each of these two categories is equal. Hence we can combine the two categories into a single category called "Computers." 

Similarly, the average of "competitive" for the categories "Antique/Art/Craft"and "Collectibles" are almost same. Therefore, we can combine them into a single category called "collectibles". The auction ending days "Sunday" and "Friday" have approximately equal averages for "competitive". So we can combine them into a single category called "Sun_Fri".

In [4]:
names(ebay.df)
ebay.df$endDay[ebay.df$endDay == "Sun"] <- "Sun_Fri"
ebay.df$endDay[ebay.df$endDay == "Fri"] <- "Sun_Fri"
ebay.df$Category[ebay.df$Category == "Business/Industrial"] <- "Computer"
ebay.df$Category[ebay.df$Category == "Antique/Art/Craft"] <- "Collectibles"


[1] "Category"     "currency"     "sellerRating" "Duration"     "endDay"      
[6] "ClosePrice"   "OpenPrice"    "Competitive."

In [5]:
head(ebay.df)

,Category,currency,sellerRating,Duration,endDay,ClosePrice,OpenPrice,Competitive.
,<chr>,<chr>,<int>,<int>,<chr>,<dbl>,<dbl>,<int>
1,Music/Movie/Game,US,3249,5,Mon,0.01,0.01,0
2,Music/Movie/Game,US,3249,5,Mon,0.01,0.01,0
3,Music/Movie/Game,US,3249,5,Mon,0.01,0.01,0
4,Music/Movie/Game,US,3249,5,Mon,0.01,0.01,0
5,Music/Movie/Game,US,3249,5,Mon,0.01,0.01,0
6,Music/Movie/Game,US,3249,5,Mon,0.01,0.01,0


Split the data into training (60%) and validation (40%) datasets. Run a logistic model with all predictors with a **cutoff of 0.5**.

In [6]:
#data partition
ntotal <- length(ebay.df$Competitive.)
ntrain <- round(ntotal * 0.6)
nvalid <- ntotal - ntrain
set.seed(202)  # Seed of random number generator for reproducibility.
ntrain.index <- sort(sample(ntotal, ntrain))  # Sample rows randomly.
train.df <- ebay.df[ntrain.index, ]
valid.df <- ebay.df[-ntrain.index, ]
options(scipen = 999, digits = 4)

#logistic regression using all the variables
reg <- glm(Competitive. ~ ., data = train.df, family = "binomial")
summary(reg)


Warning message:
“glm.fit: fitted probabilities numerically 0 or 1 occurred”



Call:
glm(formula = Competitive. ~ ., family = "binomial", data = train.df)

Deviance Residuals: 
   Min      1Q  Median      3Q     Max  
-6.813  -0.919   0.003   0.882   2.380  

Coefficients:
                               Estimate Std. Error z value             Pr(>|z|)
(Intercept)                  -0.0684576  0.5010851   -0.14               0.8913
CategoryBooks                -0.1875809  0.5045616   -0.37               0.7101
CategoryClothing/Accessories -0.7640455  0.4900804   -1.56               0.1190
CategoryCoins/Stamps         -0.6620383  0.6405184   -1.03               0.3013
CategoryCollectibles          0.6128959  0.3487541    1.76               0.0789
CategoryComputer              0.2877405  0.5093094    0.56               0.5721
CategoryElectronics           0.6004863  0.5599910    1.07               0.2836
CategoryEverythingElse       -1.4848764  1.1249556   -1.32               0.1869
CategoryHealth/Beauty        -1.5777915  0.6095667   -2.59               0.0096
Cate

Now let's run the predictions in the validation data. And after, access the accuracy buiduilg the **confusion matrix**.

In [7]:
# predictions on validation data
pred <- predict(reg, newdata = valid.df, type = "response")
head(pred)

3      4      7      8      9     10 
0.6105 0.6105 0.6105 0.6105 0.6105 0.6105

In [8]:
# confusion matrix
confusionMatrix(factor(ifelse(pred > 0.5, 1, 0)), 
                factor(valid.df$Competitive.), positive = "1")

Confusion Matrix and Statistics

          Reference
Prediction   0   1
         0 281  85
         1  80 343
                                             
               Accuracy : 0.791              
                 95% CI : (0.761, 0.819)     
    No Information Rate : 0.542              
    P-Value [Acc > NIR] : <0.0000000000000002
                                             
                  Kappa : 0.579              
                                             
 Mcnemar's Test P-Value : 0.755              
                                             
            Sensitivity : 0.801              
            Specificity : 0.778              
         Pos Pred Value : 0.811              
         Neg Pred Value : 0.768              
             Prevalence : 0.542              
         Detection Rate : 0.435              
   Detection Prevalence : 0.536              
      Balanced Accuracy : 0.790              
                                             
       'Positive

If we want to predict at the start of an auction whether it will be competitive, we cannot use the information on the `closing price`. Let's then build a logistic model **with all predictors as above, excluding price**. How does this model compare to the full model with respect to predictive accuracy?

In [9]:
#logistic regresssion excluding closing price variable
reg <- glm(Competitive. ~ ., data = train.df[-6], family = "binomial")

#predictions on validation data
pred <- predict(reg, newdata = valid.df, type = "response")
head(pred)

#confusion matrix
confusionMatrix(factor(ifelse(pred > 0.5, 1, 0)), 
                factor(valid.df$Competitive.), positive = "1")

3      4      7      8      9     10 
0.6993 0.6993 0.6993 0.6993 0.6993 0.6993

Confusion Matrix and Statistics

          Reference
Prediction   0   1
         0 201 116
         1 160 312
                                        
               Accuracy : 0.65          
                 95% CI : (0.616, 0.683)
    No Information Rate : 0.542         
    P-Value [Acc > NIR] : 0.000000000535
                                        
                  Kappa : 0.289         
                                        
 Mcnemar's Test P-Value : 0.00965       
                                        
            Sensitivity : 0.729         
            Specificity : 0.557         
         Pos Pred Value : 0.661         
         Neg Pred Value : 0.634         
             Prevalence : 0.542         
         Detection Rate : 0.395         
   Detection Prevalence : 0.598         
      Balanced Accuracy : 0.643         
                                        
       'Positive' Class : 1             
                                        

##### Conclusion:
The overall error rate for the full model (20.9%) is much lower than that for the model excluding `close price` (35%). Hence the model excluding `close price` has lower predictive accuracy compared to the full model.

Let's plot the tree, this is one of the few times we make use of the R-base graphics

### Exercise 2: Stepwise logistic regression

Use stepwise selection (use function `step()` in the `stats` package or function `stepAIC()` in the `MASS` package) to find the model with the best fit to the training data. Which predictors are used?

Aplying the function `stepAIC()` from the package `MASS` to perform a stepwise logistic regression. 

In [10]:
### BEGIN SOLUTION

# stepwise logistic regression using the function stepAIC() from the package MASS 
library(MASS)

stepwise <- stepAIC(reg, direction = "both")
summary(stepwise)

### END SOLUTION

Start:  AIC=1521
Competitive. ~ Category + currency + sellerRating + Duration + 
    endDay + OpenPrice

               Df Deviance  AIC
- OpenPrice     1     1469 1519
<none>                1469 1521
- Duration      1     1474 1524
- currency      2     1482 1530
- sellerRating  1     1483 1533
- endDay        5     1525 1567
- Category     15     1561 1583

Step:  AIC=1519
Competitive. ~ Category + currency + sellerRating + Duration + 
    endDay

               Df Deviance  AIC
<none>                1469 1519
+ OpenPrice     1     1469 1521
- Duration      1     1474 1522
- currency      2     1483 1529
- sellerRating  1     1484 1532
- endDay        5     1526 1566
- Category     15     1567 1587



Call:
glm(formula = Competitive. ~ Category + currency + sellerRating + 
    Duration + endDay, family = "binomial", data = train.df[-6])

Deviance Residuals: 
   Min      1Q  Median      3Q     Max  
-1.979  -1.080   0.699   1.018   2.326  

Coefficients:
                               Estimate Std. Error z value       Pr(>|z|)    
(Intercept)                   1.0435392  0.4064598    2.57        0.01025 *  
CategoryBooks                -0.0001252  0.4235101    0.00        0.99976    
CategoryClothing/Accessories  0.5576782  0.3667870    1.52        0.12840    
CategoryCoins/Stamps         -0.8013464  0.5520407   -1.45        0.14661    
CategoryCollectibles          1.0807095  0.2597445    4.16 0.000031732454 ***
CategoryComputer              1.0342263  0.4125733    2.51        0.01218 *  
CategoryElectronics           1.3066418  0.4420391    2.96        0.00312 ** 
CategoryEverythingElse       -1.5803968  1.1050591   -1.43        0.15267    
CategoryHealth/Beauty        -1.5345874 

#### Result:

- Using the stepwise method, the "best model" contains 24 predictors (as shown in the table above). For this model AIC=1519. 

Use **stepwise selection** to access the **new predictive error rate** (use the **validation data**). Did the accuracy increased?

In [11]:
#logistic regresssion using the selection from the stepwise process
reg_stepwise <- glm(Competitive. ~ Category + currency + sellerRating + Duration + endDay, 
                         data = train.df[-6], family = "binomial")


In [12]:
summary(reg_stepwise)


Call:
glm(formula = Competitive. ~ Category + currency + sellerRating + 
    Duration + endDay, family = "binomial", data = train.df[-6])

Deviance Residuals: 
   Min      1Q  Median      3Q     Max  
-1.979  -1.080   0.699   1.018   2.326  

Coefficients:
                               Estimate Std. Error z value       Pr(>|z|)    
(Intercept)                   1.0435392  0.4064598    2.57        0.01025 *  
CategoryBooks                -0.0001252  0.4235101    0.00        0.99976    
CategoryClothing/Accessories  0.5576782  0.3667870    1.52        0.12840    
CategoryCoins/Stamps         -0.8013464  0.5520407   -1.45        0.14661    
CategoryCollectibles          1.0807095  0.2597445    4.16 0.000031732454 ***
CategoryComputer              1.0342263  0.4125733    2.51        0.01218 *  
CategoryElectronics           1.3066418  0.4420391    2.96        0.00312 ** 
CategoryEverythingElse       -1.5803968  1.1050591   -1.43        0.15267    
CategoryHealth/Beauty        -1.5345874 

In [13]:
# predictive error rate using stepwise selection (training set)
pred <- predict(reg_stepwise, data = train.df, type = "response")

#confusion matrix
confusionMatrix(factor(ifelse(pred > 0.5, 1, 0)), 
                factor(train.df$Competitive.), positive = "1")

Confusion Matrix and Statistics

          Reference
Prediction   0   1
         0 310 153
         1 235 485
                                              
               Accuracy : 0.672               
                 95% CI : (0.644, 0.699)      
    No Information Rate : 0.539               
    P-Value [Acc > NIR] : < 0.0000000000000002
                                              
                  Kappa : 0.333               
                                              
 Mcnemar's Test P-Value : 0.0000392           
                                              
            Sensitivity : 0.760               
            Specificity : 0.569               
         Pos Pred Value : 0.674               
         Neg Pred Value : 0.670               
             Prevalence : 0.539               
         Detection Rate : 0.410               
   Detection Prevalence : 0.609               
      Balanced Accuracy : 0.664               
                                            

In [14]:
#predictive error rate using stepwise selection (Validation set)
pred <- predict(reg_stepwise, newdata = valid.df, type = "response")

#confusion matrix
confusionMatrix(factor(ifelse(pred > 0.5, 1, 0)), 
                factor(valid.df$Competitive.), positive = "1")


Confusion Matrix and Statistics

          Reference
Prediction   0   1
         0 200 116
         1 161 312
                                        
               Accuracy : 0.649         
                 95% CI : (0.614, 0.682)
    No Information Rate : 0.542         
    P-Value [Acc > NIR] : 0.000000000843
                                        
                  Kappa : 0.286         
                                        
 Mcnemar's Test P-Value : 0.0082        
                                        
            Sensitivity : 0.729         
            Specificity : 0.554         
         Pos Pred Value : 0.660         
         Neg Pred Value : 0.633         
             Prevalence : 0.542         
         Detection Rate : 0.395         
   Detection Prevalence : 0.599         
      Balanced Accuracy : 0.641         
                                        
       'Positive' Class : 1             
                                        